In [11]:
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import *
from tensorflow.keras.applications import ResNet50

In [2]:
test_dsfrom google.colab import drive
drive.mount('/content/drive')

#!tar -xzf /content/drive/MyDrive/dataset.tar.gz -C /content/drive/MyDrive

Mounted at /content/drive


In [3]:
train_dir = "/content/drive/MyDrive/Fruits-360_/data/fruits-360_100x100/fruits-360/Training"
test_dir = "/content/drive/MyDrive/Fruits-360_/data/fruits-360_100x100/fruits-360/Test"

In [10]:
train_tfms = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224),
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )
])
val_tfms = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224),
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Normalization(
        mean=[0.485, 0.456, 0.406],
        variance=[0.229**2, 0.224**2, 0.225**2]
    )
])


In [6]:
base_model = ResNet50(
    weights=None,
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.load_weights('/content/drive/MyDrive/Fruits-360_/model/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

base_model.trainable = False

In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

num_classes = len(train_ds.class_names)
print(f"Number of classes: {num_classes}")

Found 122118 files belonging to 232 classes.
Number of classes: 232


In [8]:
model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 232)            │       119,016 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,755,816 (94.44 MB)

 Trainable params: 1,168,104 (4.46 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [9]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)

train_ds = train_ds.map(lambda x, y: (train_tfms(x, training=True), y))
val_ds = test_ds.map(lambda x, y: (val_tfms(x, training=False), y))
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

Found 40715 files belonging to 232 classes.
Epoch 1/10
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 941s 242ms/step - accuracy: 0.2574 - loss: 3.3109 - val_accuracy: 0.0353 - val_loss: 29.8520
Epoch 2/10
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 952s 237ms/step - accuracy: 0.6014 - loss: 1.4098 - val_accuracy: 0.0352 - val_loss: 42.1984
Epoch 3/10
3817/3817 ━━━━━━━━━━━━━━━━━━━━ 894s 234ms/step - accuracy: 0.6727 - loss: 1.1190 - val_accuracy: 0.0253 - val_loss: 59.8228
Epoch 4/10
 535/3817 ━━━━━━━━━━━━━━━━━━━━ 10:59 201ms/step - accuracy: 0.6974 - loss: 1.0189

KeyboardInterrupt: 